In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
'''Data Manipulattion Modules'''
import numpy as np
import pandas as pd

'''Visualization Modules '''
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

'''Preprocessing libraries'''
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

'''keras libraries for CNN'''
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ReduceLROnPlateau

# **LOADING THE DATA**

In [ ]:
train=pd.read_csv('../input/Kannada-MNIST/train.csv')
test1=pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')

# **EXPLORATORY DATA ANALYSIS (EDA)**

As per data description, each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total (so each row represent one image in data set). Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

Each pixel column in the training set has a name like pixel{x}, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixel{x} is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ----------------------------------------------------------------------------------------------------------------------------------------028 029 030 031 032-----------------------------------------------------------------------------------------------------------------------------------------056

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
test1.info()

In [ ]:
test1.head()

In [ ]:
train.label.value_counts()

1. Training data set has 60000 rows and 785 columns, where as test dataset has 5000 rows and 785 columns.
2. First column is label and rest of the 784 colummns are pixel0 to pixel 783.
3. All the numbers in the dataset have equal numbers.

# **DATA PREPROCESSING**

Lets seperate features and labels of our training set.And divide our data in taining and vaildation set

In [ ]:
X_train, X_valid, y_train, y_valid =\
    train_test_split(train.drop('label', axis=1).values, train.iloc[:,0].values, 
                     test_size=0.3, random_state=1, stratify=train.iloc[:,0])

test=test1.drop('id',axis=1)
X_test = test.values

print('y_train Shape:', y_train.shape)
print('X_train Shape:', X_train.shape)
print('y_valid Shape:', y_valid.shape)
print('X_valid Shape:', X_valid.shape)
print('X_test Shape: ', X_test.shape)

Next, we need to reshape our dataset inputs (Xs_train and Xs_test) to the shape that our model expects when we train the model. The first number is the number of images (-1). Then comes the shape of each image (28x28). The last number is 1, which signifies that the images are greyscale. if 3 implies RGB mode

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

print('X_train Shape:', X_train.shape)
print('X_valid Shape:', X_valid.shape)
print('X_test Shape: ', X_test.shape)

# Plot the Images

In [ ]:
plt.imshow(X_train[0][:,:,0])

In [ ]:
plt.imshow(X_train[10][:,:,0])

____________

Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. Therefore, we need to scale pixel values between 0 and 1. 

In [ ]:
Xs_train = X_train / 255
Xs_valid = X_valid / 255

ENCODING TARGET VALUES.The categorical value assignment can be done using sklearn’s LabelEncoder. Problem with label encoding is that it assumes higher the categorical value, better the category, WHICH WE DONT WANT! lets use Keras inbuild library to_categorical() for one-hot encoding. 

In [ ]:
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)

# **BUILDING MODEL**

In [ ]:
np.random.seed(42)

#The Sequential model is a linear stack of layers.

from keras.models import Sequential

# Activation the Model
Kannad_cnn = Sequential()

# adding layers to model
#BLOCK 1
Kannad_cnn.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28,1), activation = 'relu') )
Kannad_cnn.add(MaxPool2D(2,2))
Kannad_cnn.add(BatchNormalization())

#BLOCK 2
Kannad_cnn.add(Conv2D(64, (3,3), padding='same', activation = 'relu') )
Kannad_cnn.add(MaxPool2D(2,2))
Kannad_cnn.add(BatchNormalization())

#Final Block
Kannad_cnn.add(Flatten())
Kannad_cnn.add(Dense(64, activation='relu'))
Kannad_cnn.add(BatchNormalization())

#In between the convolutional layer and the fully connected layer, there is a ‘Flatten’ layer. Flattening transforms 
#a two-dimensional matrix of features into a vector that can be fed into a fully connected neural network classifier.

Kannad_cnn.add(Dense(10, activation='softmax'))

Kannad_cnn.summary()

Optimizers update the weight parameters to minimize the loss function. Loss function acts as guides to the terrain telling optimizer if it is moving in the right direction to reach the bottom of the valley, the global minimum.

In [ ]:
optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08)

In [ ]:
Kannad_cnn.compile(optimizer=optimizer,loss=['categorical_crossentropy'],metrics=['accuracy'])

In [ ]:
h1 = Kannad_cnn.fit(Xs_train, y_train, batch_size=1024, epochs=40, verbose=1, 
                   validation_data = [Xs_valid, y_valid]) 

In [ ]:
plt.figure(figsize=[10,5])
plt.subplot(1,2,1)
plt.plot(range(1,41), h1.history['accuracy'], label='Training Accuracy')
plt.plot(range(1,41), h1.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(1,41), h1.history['loss'], label='Training Loss')
plt.plot(range(1,41), h1.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()

In [ ]:
Y_test = Kannad_cnn.predict_classes(X_test)

In [ ]:
my_submission = pd.DataFrame({'id' :test1. iloc[:,0], 'label':Y_test })

print(my_submission.iloc[:5])

my_submission.to_csv('submission.csv', index=False)